In [2]:
import pandas as pd
import numpy as np
import itertools
import datetime
import matplotlib.pyplot as plt
from datetime import *
from datetime import datetime, timedelta, date
%load_ext google.colab.data_table
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Standard plotly imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [5]:
#Change the date to the previous day
query = """
WITH Schedule as
(
SELECT
EXTRACT(DATE FROM Session5Date) Date, Year, RoundNumber, EventName, EventFormat
FROM `perceptive-ivy-290216.f1_api.schedules`
WHERE Session5='Race'
GROUP BY 1,2,3,4,5
ORDER BY 2,1
)
,

Results as (
SELECT
DriverNumber, Abbreviation, TeamName,FullName, CAST(Position as INT64) Position,CAST(GridPosition as INT64) GridPosition, Time, Status, CAST(Points as INT64) Points, A.Year, GP, B.Date, B.RoundNumber, B.EventFormat
FROM `perceptive-ivy-290216.f1_api.results_race` A
JOIN schedule B
ON A.GP=B.EventName AND A.Year=B.Year
# WHERE A.Year=2016
ORDER BY FullName, RoundNumber
)

SELECT
DriverNumber, Abbreviation, TeamName,FullName, Position, GridPosition, Time, Status, Points,
SUM(Points) OVER (PARTITION BY Year, FullName ORDER BY ROUNDNUMBER) PTS,
Year, GP, Date, RoundNumber, EventFormat
FROM
Results
ORDER BY FullName, RoundNumber

"""
project_id = "perceptive-ivy-290216"
df_bq = pd.read_gbq(query=query, project_id=project_id, dialect='standard')

In [6]:
df_bq_driver=df_bq[df_bq["Year"]==2021]
df_bq_driver

,DriverNumber,Abbreviation,TeamName,FullName,Position,GridPosition,Time,Status,Points,PTS,Year,GP,Date,RoundNumber,EventFormat
1392,99,None,Alfa Romeo Racing,Antonio Giovinazzi,12,12,NaT,+1 Lap,0,0,2021,Bahrain Grand Prix,2021-03-28,1,conventional
1396,99,None,Alfa Romeo Racing,Antonio Giovinazzi,14,17,NaT,+1 Lap,0,0,2021,Emilia Romagna Grand Prix,2021-04-18,2,conventional
1399,99,None,Alfa Romeo Racing,Antonio Giovinazzi,12,12,NaT,+1 Lap,0,0,2021,Portuguese Grand Prix,2021-05-02,3,conventional
1402,99,None,Alfa Romeo Racing,Antonio Giovinazzi,15,14,NaT,+1 Lap,0,0,2021,Spanish Grand Prix,2021-05-09,4,conventional
1405,99,None,Alfa Romeo Racing,Antonio Giovinazzi,10,10,NaT,+1 Lap,1,1,2021,Monaco Grand Prix,2021-05-23,5,conventional
1408,99,None,Alfa Romeo Racing,Antonio Giovinazzi,11,20,0 days 02:13:46.664000,Finished,0,1,2021,Azerbaijan Grand Prix,2021-06-06,6,conventional
1411,99,None,Alfa Romeo Racing,Antonio Giovinazzi,15,13,NaT,+1 Lap,0,1,2021,French Grand Prix,2021-06-20,7,conventional
1414,99,None,Alfa Romeo Racing,Antonio Giovinazzi,15,15,NaT,+1 Lap,0,1,2021,Styrian Grand Prix,2021-06-27,8,conventional
1417,99,None,Alfa Romeo Racing,Antonio Giovinazzi,14,15,NaT,+1 Lap,0,1,2021,Austrian Grand Prix,2021-07-04,9,conventional
1420,99,None,Alfa Romeo Racing,Antonio Giovinazzi,13,15,NaT,+1 Lap,0,1,2021,British Grand Prix,2021-07-18,10,sprint


In [10]:
px.line(df_bq_driver, x="Date", y="PTS", color='FullName', markers=True,
        hover_data=['DriverNumber','TeamName', 'Position', 'GridPosition', 'Status', 'Year', 'GP', 'RoundNumber'], height=1000, width=2000,

            color_discrete_map={
                 "HAM": "#6CD3BF",
                 "VER": "#3860C6",
                 "OCO": "#2293D1",
                 "NOR": "#F58020",
                 "MAG": "#B6BABD",
                 "ALB": "#37BEDD",
                 "BOT": "#C92D4B",
                 "LEC": "#F91536",
                 "VET": "#358C75",
                 "GAS": "#5E8FAA",
                 "RUS": "#6CD3BF",
                 "PER": "#3860C6",
                 "ALO": "#2293D1",
                 "RIC": "#F58020",
                 "MSC": "#B6BABD",
                 "LAT": "#37BEDD",
                 "ZHO": "#C92D4B",
                 "SAI": "#F91536",
                 "STR": "#358C75",
                 "TSU": "#5E8FAA"
                 }
        )

In [8]:
df_team=df_bq[df_bq["Year"]==2021]
df_team=df_team.groupby(by=["TeamName","Date","GP","Year","EventFormat","RoundNumber"])["Points","PTS"].sum()
df_team=pd.DataFrame(df_team)
df_team.reset_index(inplace=True)
df_team.head()

<ipython-input-8-1ee36737622f>:2: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



,TeamName,Date,GP,Year,EventFormat,RoundNumber,Points,PTS
0,Alfa Romeo Racing,2021-03-28,Bahrain Grand Prix,2021,conventional,1,0,0
1,Alfa Romeo Racing,2021-04-18,Emilia Romagna Grand Prix,2021,conventional,2,0,0
2,Alfa Romeo Racing,2021-05-02,Portuguese Grand Prix,2021,conventional,3,0,0
3,Alfa Romeo Racing,2021-05-09,Spanish Grand Prix,2021,conventional,4,0,0
4,Alfa Romeo Racing,2021-05-23,Monaco Grand Prix,2021,conventional,5,1,1


In [9]:
px.line(df_team, x="Date", y="PTS", color='TeamName', markers=True,
        hover_data=['TeamName', 'Date', 'GP', 'Year', 'EventFormat', 'RoundNumber'], height=1000, width=2000,
        color_discrete_map={
                 "Mercedes": "#6CD3BF",
                 "Red Bull Racing": "#3671C6",
                 "Alpine": "#2293D1",
                 "McLaren": "#FFA500",
                 "Haas": "#B6BABD",
                 "Williams": "#37BEDD",
                 "Alfa Romeo Racing": "#C92D4B",
                 "Ferrari": "#F91536",
                 "Aston Martin": "#358C75",
                 "AlphaTauri": "#5E8FAA" }
        )